In [24]:
import os
import random
from PIL import Image, ImageOps

def generate_face_pairs(passive_eyes_files, mouths_files, eyebrows_files, 
                        num_pairs=1, output_dir="output"):
    """
    Generate pairs of passive and active faces with random eye configurations (1-4 eyes).
    Active eyes are created from passive eyes.
    
    Args:
        passive_eyes_files (list): List of file paths to passive eye images
        mouths_files (list): List of file paths to mouth images
        eyebrows_files (list): List of file paths to eyebrow images (single eyebrow)
        num_pairs (int): Number of face pairs to generate
        output_dir (str): Directory to save the generated faces
        
    Returns:
        list: List of tuples containing paths to the generated (passive, active) face pairs
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    result_pairs = []
    
    for i in range(num_pairs):
        # Create a unique ID for this face pair
        face_id = random.randint(10000, 99999)
        
        # Create blank transparent images (RGBA mode)
        passive_face = Image.new('RGBA', (100, 100), (0, 0, 0, 0))
        active_face = Image.new('RGBA', (100, 100), (0, 0, 0, 0))
        
        # Randomly select number of eyes (1, 2, 3, or 4)
        num_eyes = random.choice([1, 2, 3, 4])
        
        # Randomly select assets for this pair
        passive_eye_path = random.choice(passive_eyes_files)
        mouth_path = random.choice(mouths_files)
        eyebrow_path = random.choice(eyebrows_files)
        
        # Load the selected assets
        passive_eye_original = Image.open(passive_eye_path).convert('RGBA')
        mouth = Image.open(mouth_path).convert('RGBA')
        eyebrow_original = Image.open(eyebrow_path).convert('RGBA')
        
        # Crop assets to remove transparent background
        passive_eye = crop_transparent(passive_eye_original)
        eyebrow = crop_transparent(eyebrow_original)
        
        # Resize assets if needed (maintaining aspect ratio)
        eye_width = 20  # Adjust as needed
        # For multiple eyes, make them smaller
        if num_eyes > 2:
            eye_width = 15
        passive_eye = resize_image(passive_eye, eye_width)
        
        mouth_width = 40  # Adjust as needed
        mouth = resize_image(mouth, mouth_width)
        
        eyebrow_width = 20  # Adjust as needed
        if num_eyes > 2:
            eyebrow_width = 15
        eyebrow = resize_image(eyebrow, eyebrow_width)
        
        # Create active eye by cutting passive eye in half (keeping top half)
        active_eye = passive_eye.crop((0, 0, passive_eye.width, passive_eye.height // 2))
        
        # Create mirrored eyebrow for right side (horizontal mirror only)
        mirrored_eyebrow = ImageOps.mirror(eyebrow)
        
        # Define base positions
        center_x = 50
        passive_eye_y = 45
        mouth_x, mouth_y = 50, 70
        
        # Generate a single random value for eyebrow height variation
        eyebrow_offset = random.randint(0, 5)
        
        # Calculate active eye position, ensuring lower bound matches
        active_eye_y = passive_eye_y - (passive_eye.height - active_eye.height)
        
        # Create passive and active faces based on eye configuration
        if num_eyes == 1:  # Single eye in the middle
            # Place single eye in the center
            paste_centered(passive_face, passive_eye, center_x, passive_eye_y)
            paste_centered(active_face, active_eye, center_x, active_eye_y)
            
            # Calculate eyebrow positions for middle eye (create a unibrow)
            eyebrow_y = passive_eye_y - eyebrow.height + 8 - eyebrow_offset
            
            # Create a unibrow by overlapping mirrored eyebrows in the middle
            # Position them slightly offset from center
            left_brow_x = center_x - 5
            right_brow_x = center_x + 5
            
            # For active face only
            paste_centered(active_face, eyebrow, left_brow_x, eyebrow_y)
            paste_centered(active_face, mirrored_eyebrow, right_brow_x, eyebrow_y)
            
        elif num_eyes == 2:  # Standard two eyes
            # Calculate positions for two eyes
            left_eye_x, right_eye_x = center_x - 20, center_x + 20
            
            # Place two eyes
            paste_centered(passive_face, passive_eye, left_eye_x, passive_eye_y)
            paste_centered(passive_face, passive_eye, right_eye_x, passive_eye_y)
            paste_centered(active_face, active_eye, left_eye_x, active_eye_y)
            paste_centered(active_face, active_eye, right_eye_x, active_eye_y)
            
            # Calculate eyebrow positions
            eyebrow_y = passive_eye_y - eyebrow.height + 8 - eyebrow_offset
            
            # Place eyebrows for active face
            paste_centered(active_face, eyebrow, left_eye_x, eyebrow_y)
            paste_centered(active_face, mirrored_eyebrow, right_eye_x, eyebrow_y)
            
        elif num_eyes == 3:  # Three eyes (two on sides, one in middle)
            # Calculate positions for three eyes
            left_eye_x = center_x - 25
            middle_eye_x = center_x
            right_eye_x = center_x + 25
            
            # Place three eyes
            paste_centered(passive_face, passive_eye, left_eye_x, passive_eye_y)
            paste_centered(passive_face, passive_eye, middle_eye_x, passive_eye_y)
            paste_centered(passive_face, passive_eye, right_eye_x, passive_eye_y)
            paste_centered(active_face, active_eye, left_eye_x, active_eye_y)
            paste_centered(active_face, active_eye, middle_eye_x, active_eye_y)
            paste_centered(active_face, active_eye, right_eye_x, active_eye_y)
            
            # Calculate eyebrow positions
            eyebrow_y = passive_eye_y - eyebrow.height + 8 - eyebrow_offset
            
            # Place eyebrows for active face - outer eyes
            paste_centered(active_face, eyebrow, left_eye_x, eyebrow_y)
            paste_centered(active_face, mirrored_eyebrow, right_eye_x, eyebrow_y)
            
            # Create a smaller unibrow for middle eye
            middle_left_brow_x = middle_eye_x - 3
            middle_right_brow_x = middle_eye_x + 3
            middle_eyebrow = resize_image(eyebrow, eyebrow_width * 0.7)
            middle_mirrored_eyebrow = resize_image(mirrored_eyebrow, eyebrow_width * 0.7)
            
            # Place middle unibrow
            paste_centered(active_face, middle_eyebrow, middle_left_brow_x, eyebrow_y)
            paste_centered(active_face, middle_mirrored_eyebrow, middle_right_brow_x, eyebrow_y)
            
        else:  # Four eyes (two rows of two)
            # Calculate positions for four eyes
            upper_y = passive_eye_y - 10
            lower_y = passive_eye_y + 10
            left_x = center_x - 15
            right_x = center_x + 15
            
            # Calculate active eye positions
            upper_active_y = upper_y - passive_eye.height + 8
            lower_active_y = lower_y - passive_eye.height + 8
            
            # Place four eyes
            paste_centered(passive_face, passive_eye, left_x, upper_y)
            paste_centered(passive_face, passive_eye, right_x, upper_y)
            paste_centered(passive_face, passive_eye, left_x, lower_y)
            paste_centered(passive_face, passive_eye, right_x, lower_y)
            
            paste_centered(active_face, active_eye, left_x, upper_active_y)
            paste_centered(active_face, active_eye, right_x, upper_active_y)
            paste_centered(active_face, active_eye, left_x, lower_active_y)
            paste_centered(active_face, active_eye, right_x, lower_active_y)
            
            # Calculate eyebrow positions (only above upper eyes)
            eyebrow_y = upper_y - eyebrow.height - 2 - eyebrow_offset
            
            # Place eyebrows for active face
            paste_centered(active_face, eyebrow, left_x, eyebrow_y)
            paste_centered(active_face, mirrored_eyebrow, right_x, eyebrow_y)
        
        # Place mouth on both faces
        paste_centered(passive_face, mouth, mouth_x, mouth_y)
        paste_centered(active_face, mouth, mouth_x, mouth_y)
        
        # Save the generated faces
        passive_face_path = os.path.join(output_dir, f"passive_face_{face_id}_{num_eyes}eyes.png")
        active_face_path = os.path.join(output_dir, f"active_face_{face_id}_{num_eyes}eyes.png")
        
        passive_face.save(passive_face_path)
        active_face.save(active_face_path)
        
        # Add to results
        result_pairs.append((passive_face_path, active_face_path))
    
    return result_pairs

def crop_transparent(image):
    """Crop image to remove transparent border/background"""
    # Get the alpha channel
    alpha = image.getchannel('A')
    
    # Get the bounding box of the non-zero regions (non-transparent)
    bbox = alpha.getbbox()
    
    if bbox:
        # Crop the image to the bounding box
        return image.crop(bbox)
    else:
        # If the image is completely transparent, return the original
        return image

def paste_centered(base_image, overlay_image, x_center, y_center):
    """Paste an image centered at the specified coordinates"""
    x = x_center - overlay_image.width // 2
    y = y_center - overlay_image.height // 2
    base_image.paste(overlay_image, (x, y), overlay_image)

def resize_image(image, target_width):
    """Resize image to target width while maintaining aspect ratio"""
    w, h = image.size
    target_height = int(h * target_width / w)
    return image.resize((target_width, target_height), Image.LANCZOS)

# Example usage:
if __name__ == "__main__":
    # Example lists of file paths (you would provide actual file paths)
    passive_eyes_files = [
        "./FaceGenerationAssets/PassiveEyes/eye_1.png",
        "./FaceGenerationAssets/PassiveEyes/eye_2.png",
        "./FaceGenerationAssets/PassiveEyes/eye_3.png",
        "./FaceGenerationAssets/PassiveEyes/eye_4.png",
        "./FaceGenerationAssets/PassiveEyes/eye_5.png"
    ]

    mouths_files = [
        "./FaceGenerationAssets/Mouths/mouth_1.png",
        "./FaceGenerationAssets/Mouths/mouth_2.png",
        "./FaceGenerationAssets/Mouths/mouth_3.png",
        "./FaceGenerationAssets/Mouths/mouth_4.png",
        "./FaceGenerationAssets/Mouths/mouth_5.png"
    ]
    
    eyebrows_files = [
        "./FaceGenerationAssets/Eyebrows/eyebrow_1.png",
        "./FaceGenerationAssets/Eyebrows/eyebrow_2.png",
        "./FaceGenerationAssets/Eyebrows/eyebrow_3.png",
    ]
    
    # Generate 5 face pairs
    face_pairs = generate_face_pairs(
        passive_eyes_files,
        mouths_files,
        eyebrows_files,
        num_pairs=5,
        output_dir="./FaceGenerationAssets/Results"
    )
    
    print(f"Generated {len(face_pairs)} face pairs:")
    for passive, active in face_pairs:
        print(f"Passive: {passive}, Active: {active}")

TypeError: 'float' object cannot be interpreted as an integer